### dev_notebook_

This is your development environment...you may start:
- Exploring your data.
- Testing the ad-hoc methods.
- Designing your pipeline.

In [132]:
import pandas as pd 
import requests as req
import json
import re

#%pip install shapely
#%pip install geopandas
from shapely.geometry import Point
import geopandas as gpd

#### **API Consulados&Embajadas**

In [133]:
#call the API of the EMT de Madrir
url = 'https://datos.madrid.es/egob'
veronica='/catalogo/201000-0-embajadas-consulados.json'
url=url + veronica

res = req.get(url)  
res

<Response [200]>

In [134]:
#json
data=res.json()
#data

In [135]:
data.keys()

dict_keys(['@context', '@graph'])

In [136]:
graph=data['@graph']

In [137]:
#function to find which embassies already have data from nearby bicimad stations
def find_element_with_word(lista, word):
    for element in lista:
        if word in element:
            return element
        else:
            pass
            
    return 'No tiene Bicimad'
    
   

In [138]:
#apply it to all rows
org=[]
tem_list=[]    
for dic in graph:
    listas=dic['organization']['organization-desc'].split('.  ') 
    tem_list.append(listas)
for lista in tem_list:
    e=find_element_with_word(lista,'Bicimad')
    org.append(e)
           
#org


In [139]:
#separate latitude and longitude
dp=pd.DataFrame(graph)
location = pd.DataFrame(pd.json_normalize(dp['location']))
location

,latitude,longitude
0,40.428216,-3.692666
1,40.453376,-3.680978
2,40.442447,-3.679665
3,40.425716,-3.688868
4,40.428626,-3.689004
...,...,...
152,40.457056,-3.690768
153,40.462675,-3.687881
154,40.472005,-3.770871
155,40.466107,-3.666833


In [140]:
#take the info from address
adress= pd.DataFrame(pd.json_normalize(dp['address']))
adress.head()

,locality,postal-code,street-address,district.@id,area.@id
0,MADRID,28010,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
1,MADRID,28016,CALLE TALAVERA 9,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
2,MADRID,28002,CALLE AVIADOR LINDBERGH 3,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
3,MADRID,28001,CALLE GOYA 5 y 7 pasaje,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...
4,MADRID,28046,PASEO CASTELLANA 18 PLANTA 6 Izquierda,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...


In [141]:
#concatenate the tables to decide which data we want to use
embajadas=pd.concat([dp, adress, location], axis=1).drop(columns=['id','@id','@type','relation','address', 'organization','district.@id','area.@id','location','locality','postal-code'])
embajadas['number']=org
embajadas['type_of_place']='Embajadas y Consulados'
embajadas.head()

,title,street-address,latitude,longitude,number,type_of_place
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,Bicimad: Estaciones 8 (plaza de Alonso Martíne...,Embajadas y Consulados
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,"Bicimad: Estaciones 148 (calle Serrano, 210), ...",Embajadas y Consulados
2,Consulado de Bolivia,CALLE AVIADOR LINDBERGH 3,40.442447,-3.679665,"Bicimad: Estación 146 (calle María Francisca, 1).",Embajadas y Consulados
3,Consulado de Brasil,CALLE GOYA 5 y 7 pasaje,40.425716,-3.688868,"Bicimad: Estaciones 93 (calle Goya, 1) y 106 a...",Embajadas y Consulados
4,Consulado de Bélgica,PASEO CASTELLANA 18 PLANTA 6 Izquierda,40.428626,-3.689004,Bicimad: Estaciones 104 (paseo de la Castellan...,Embajadas y Consulados


In [142]:
#separate the stations near each embassy and return a list with just the numbers

celdas = []

for celda in embajadas['number']:

    if ') y ' in celda:
        celda=celda.replace(') y', '), ')   #unify the format to be able to separate the string 
    
    
    if 'a y b' in celda: #number of stations option
        ayb_list=[] 
         
        if 'Bicimad: Estaciones ' in celda:
            num = celda.split('Bicimad: Estaciones')[1].split('),')         
            for e in num:                                
                if 'a y b' in e:
                    nums=re.findall(r'\b(\d+)\s*a\s*y\s*b\b', e)
                    ayb_list.append(f'{nums[0]}a')
                    ayb_list.append(f'{nums[0]}b')
                else: 
                    ayb_list.append(re.findall('\s\d+\s', e)[0])
            celdas.append(ayb_list)
                
        elif 'Bicimad: Estación ' in celda:
            nums=re.findall('\s\d+\s', celda)
            celdas.append([f'{nums[0].strip()}a', f'{nums[0].strip()}b'])
        
    if 'a y b' not in celda:
        if 'Bicimad: Estaciones ' in celda:
            partes = re.findall('\s\d+\s', celda)
            celdas.append(partes)
        elif 'Bicimad: Estación ' in celda:
            parte = re.findall('\s\d+\s', celda)[0]
            celdas.append(parte)
        else:
            celdas.append('0') #where there is no data
#celdas

In [143]:
embajadas['number']=celdas


**API BONUS**

In [144]:
#connect to the EMT de madrid API to obtain my accessToken
url = 'https://openapi.emtmadrid.es/v1/mobilitylabs/user/login/'

headers = {
    'email': 'veronicagrim@hotmail.com',
    'password': 'Proyectodata14'
    }

response = req.request('GET', url, headers=headers)
response.json()

{'code': '01',
 'description': 'Token extend  into control-cache Data recovered  OK, (lapsed: 192 millsecs)',
 'datetime': '2024-03-11T18:38:41.384881',
 'data': [{'nameApp': 'OPENAPI MobilityLabs',
   'levelApp': 0,
   'updatedAt': '2024-03-05T18:29:05.2530000',
   'userName': 'veronicagrim@hotmail.com',
   'lastUpdate': {'$date': 1710173323605},
   'idUser': '2c4e1717-b9d6-4bcf-b026-38157bb8f83b',
   'priv': 'U',
   'tokenSecExpiration': 86399,
   'email': 'veronicagrim@hotmail.com',
   'tokenDteExpiration': {'$date': 1710263322605},
   'flagAdvise': True,
   'accessToken': 'f80fe3d1-a76b-452b-bc7e-21302f7228da',
   'apiCounter': {'current': 47,
    'dailyUse': 20000,
    'owner': 0,
    'licenceUse': 'Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!',
    'aboutUses': 'If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylab

In [145]:
#get the accessToken
accessToken=response.json()['data'][0]['accessToken']

In [146]:
#connect to the EMT de madrid API to obtain data from all bicimad stations.
url='https://openapi.emtmadrid.es/v1/transport/bicimad/stations/'

headers = {
    'accessToken': accessToken,
    }

response = req.request("GET", url, headers=headers)
estaciones=response.json()

In [147]:
estaciones.keys()

dict_keys(['code', 'description', 'datetime', 'data'])

In [148]:
estaciones=estaciones['data']
estaciones_api=pd.DataFrame(estaciones)


In [149]:
#clean name and separate latitude and longitude
names=[f['name'].split('-')[1].strip() for f in estaciones]
coordenadas=[[f['geometry']['coordinates'] for f in estaciones]]
coordenadas

estaciones_api['name']=names
estaciones_api['geometry']=coordenadas[0]  
estaciones_api

,activate,address,dock_bikes,free_bases,geofenced_capacity,geometry,id,light,name,no_available,number,reservations_count,total_bases,virtualDelete
0,1,"Calle Miguel Moya nº 1,",10,17,20,"[-3.70569, 40.4204]",1406,2,Metro Callao,0,2,0,27,False
1,1,"Plaza Conde Surchill, 4 ,",4,15,20,"[-3.7072537, 40.4303223]",1407,0,Plaza Conde Suchil,0,3,0,19,False
2,1,"Calle Fuencarral nº 106,",23,3,20,"[-3.7021354, 40.4285212]",1409,1,Fuencarral,0,5,0,27,False
3,1,"Calle Hortaleza nº 63,",10,9,20,"[-3.698447, 40.424148]",1410,2,Colegio Arquitectos,0,6,0,19,False
4,1,"Calle Hortaleza nº 75,",19,0,20,"[-3.6977715, 40.4251906]",1411,1,Hortaleza,0,7,0,19,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,1,Avenida Menéndez pelayo (esquina con calle Poe...,26,1,20,"[-3.6784749, 40.4082717]",1477,1,Parque del Retiro,0,75,0,27,False
607,1,"Calle Joaquin Turina, 49,",8,19,20,"[-3.76182, 40.372458]",2145,0,"Joaquin Turina, 49",0,371,0,27,False
608,1,"Campuzano, 1,",8,15,20,"[-3.6423386, 40.4356873]",1965,2,Metro Pueblo Nuevo,0,482,0,23,False
609,1,"Calle de las Delicias, 41 ,",1,18,20,"[-3.69091831, 40.40381914]",2117,0,Delicias,0,273,0,19,False


In [150]:
estaciones_api=estaciones_api[['name', 'number', 'address','geometry','activate','no_available','total_bases','dock_bikes','free_bases']]
estaciones_api

,name,number,address,geometry,activate,no_available,total_bases,dock_bikes,free_bases
0,Metro Callao,2,"Calle Miguel Moya nº 1,","[-3.70569, 40.4204]",1,0,27,10,17
1,Plaza Conde Suchil,3,"Plaza Conde Surchill, 4 ,","[-3.7072537, 40.4303223]",1,0,19,4,15
2,Fuencarral,5,"Calle Fuencarral nº 106,","[-3.7021354, 40.4285212]",1,0,27,23,3
3,Colegio Arquitectos,6,"Calle Hortaleza nº 63,","[-3.698447, 40.424148]",1,0,19,10,9
4,Hortaleza,7,"Calle Hortaleza nº 75,","[-3.6977715, 40.4251906]",1,0,19,19,0
...,...,...,...,...,...,...,...,...,...
606,Parque del Retiro,75,Avenida Menéndez pelayo (esquina con calle Poe...,"[-3.6784749, 40.4082717]",1,0,27,26,1
607,"Joaquin Turina, 49",371,"Calle Joaquin Turina, 49,","[-3.76182, 40.372458]",1,0,27,8,19
608,Metro Pueblo Nuevo,482,"Campuzano, 1,","[-3.6423386, 40.4356873]",1,0,23,8,15
609,Delicias,273,"Calle de las Delicias, 41 ,","[-3.69091831, 40.40381914]",1,0,19,1,18


In [151]:
#separate latitude and longitude
def dividir_lista(row):
    return pd.Series(row['geometry'])
estaciones_api[['longitude', 'latitude']] = estaciones_api.apply(dividir_lista, axis=1)
estaciones_api=estaciones_api.drop(columns=['geometry'])

C:\Users\Veronica\AppData\Local\Temp\ipykernel_25604\2413765795.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estaciones_api[['longitude', 'latitude']] = estaciones_api.apply(dividir_lista, axis=1)
C:\Users\Veronica\AppData\Local\Temp\ipykernel_25604\2413765795.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estaciones_api[['longitude', 'latitude']] = estaciones_api.apply(dividir_lista, axis=1)


In [152]:
estaciones_api.head(11)

,name,number,address,activate,no_available,total_bases,dock_bikes,free_bases,longitude,latitude
0,Metro Callao,2,"Calle Miguel Moya nº 1,",1,0,27,10,17,-3.705690,40.420400
1,Plaza Conde Suchil,3,"Plaza Conde Surchill, 4 ,",1,0,19,4,15,-3.707254,40.430322
2,Fuencarral,5,"Calle Fuencarral nº 106,",1,0,27,23,3,-3.702135,40.428521
3,Colegio Arquitectos,6,"Calle Hortaleza nº 63,",1,0,19,10,9,-3.698447,40.424148
4,Hortaleza,7,"Calle Hortaleza nº 75,",1,0,19,19,0,-3.697771,40.425191
5,Metro Alonso Martínez,8,"Plaza de Alonso Martínez nº 6,",1,0,24,7,15,-3.695440,40.427868
6,Plaza de San Miguel,9,"Plaza de San Miguel nº 10,",1,0,24,8,16,-3.709508,40.415606
7,Marqués de la Ensenada,10,"Calle Marqués de la Ensenada (junto a edif, Ce...",1,0,23,16,7,-3.692104,40.425403
8,Plaza del Dos de Mayo,11,"Calle San Andrés nº 18 - 20,",1,0,23,4,19,-3.703600,40.427000
9,San Hermenegildo,12,"Calle San Bernardo nº 85 - 87,",1,0,27,8,19,-3.706141,40.428467


In [154]:
#function to measure the distance
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair points in degrees 
    # (e.g.: Start Point -> 40.4400607 / -3.6425358 End Point -> 40.4234825 / -3.6292625)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [156]:
#
def closest_distances(embajadas, estaciones_api):
    
    distances = []
    for index, row in estaciones_api.iterrows():
        lat_finish = row['latitude']
        long_finish = row['longitude']
        
        distance = distance_meters(embajadas['latitude'] , embajadas['longitude'], lat_finish, long_finish)
        distances.append([row['number'],distance[0]])
       

    distances.sort(key=lambda x: x[1])

    #return ([station[0] for station in distances[:3]])
    return ([station[1] for station in distances[:3]])
    #return distances[:3]

In [157]:
estaciones_bicimad=[]
for index, row in embajadas.iterrows():
    estaciones_bicimad.append(closest_station(row, estaciones_api))
    
distance=[]
for index, row in embajadas.iterrows():
    distance.append(closest_distances(row, estaciones_api))

In [161]:
len(estaciones_bicimad)

157

In [162]:
distance

[[311.9001139109231, 319.3811255516, 380.0258425515077],
 [255.76551654772592, 312.6027620662095, 394.27088893706156],
 [220.77884425921926, 533.6066251773035, 558.6140486609512],
 [52.439373963638864, 141.64985143796721, 147.60749201069927],
 [216.3270282884061, 236.7967821117334, 321.2985998082369],
 [172.40063434707065, 523.9140628883374, 540.0406853334919],
 [257.18864309012383, 278.7003102221015, 301.54739270170205],
 [293.0259214512717, 307.66642504405394, 463.4864226407781],
 [266.5192373383566, 431.88223067626967, 584.8398661593218],
 [148.69877968455137, 372.6422286451238, 509.9351113407207],
 [282.9030513501236, 348.4052941514271, 474.40697535851194],
 [56.80763250489319, 277.9016033102922, 280.997593676709],
 [106.38951058709964, 230.0065043117107, 245.37217484577025],
 [253.8620770258948, 313.6596068959931, 429.59570995471825],
 [nan, nan, nan],
 [188.94848445324814, 245.73830310623472, 302.7756283193919],
 [129.44161209565095, 420.72699653801055, 430.603719084309],
 [369.1

In [163]:
distances=[d for e in distance for d in e]
distances  

[311.9001139109231,
 319.3811255516,
 380.0258425515077,
 255.76551654772592,
 312.6027620662095,
 394.27088893706156,
 220.77884425921926,
 533.6066251773035,
 558.6140486609512,
 52.439373963638864,
 141.64985143796721,
 147.60749201069927,
 216.3270282884061,
 236.7967821117334,
 321.2985998082369,
 172.40063434707065,
 523.9140628883374,
 540.0406853334919,
 257.18864309012383,
 278.7003102221015,
 301.54739270170205,
 293.0259214512717,
 307.66642504405394,
 463.4864226407781,
 266.5192373383566,
 431.88223067626967,
 584.8398661593218,
 148.69877968455137,
 372.6422286451238,
 509.9351113407207,
 282.9030513501236,
 348.4052941514271,
 474.40697535851194,
 56.80763250489319,
 277.9016033102922,
 280.997593676709,
 106.38951058709964,
 230.0065043117107,
 245.37217484577025,
 253.8620770258948,
 313.6596068959931,
 429.59570995471825,
 nan,
 nan,
 nan,
 188.94848445324814,
 245.73830310623472,
 302.7756283193919,
 129.44161209565095,
 420.72699653801055,
 430.603719084309,
 369.12

In [164]:
embajadas['number']=estaciones_bicimad
embajadas=embajadas.explode('number')
embajadas['distance']=distances
embajadas

,title,street-address,latitude,longitude,number,type_of_place,distance
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,Embajadas y Consulados,311.900114
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,Embajadas y Consulados,319.381126
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,Embajadas y Consulados,380.025843
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,Embajadas y Consulados,255.765517
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,Embajadas y Consulados,312.602762
...,...,...,...,...,...,...,...
155,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,Embajadas y Consulados,422.553954
155,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,Embajadas y Consulados,537.184597
156,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,Embajadas y Consulados,179.924919
156,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,Embajadas y Consulados,291.887270


In [165]:
for i, row in embajadas.iterrows():
    if 'Consulado' in row['title']:
        embajadas.at[i, 'type_of_place'] = 'consulado'
    else:
        embajadas.at[i, 'type_of_place'] = 'embajada'
        
embajadas

,title,street-address,latitude,longitude,number,type_of_place,distance
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,consulado,311.900114
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,consulado,319.381126
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,consulado,380.025843
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,consulado,255.765517
1,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,consulado,312.602762
...,...,...,...,...,...,...,...
155,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,embajada,422.553954
155,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,embajada,537.184597
156,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,embajada,179.924919
156,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,embajada,291.887270


In [166]:
embajadas.to_csv('../data/processed/embajadas.csv', index=False)

In [168]:
result = pd.merge(embajadas, estaciones_api, on='number', how='left')
result
#result.sort_values(by='title')

,title,street-address,latitude_x,longitude_x,number,type_of_place,distance,name,address,activate,no_available,total_bases,dock_bikes,free_bases,longitude_y,latitude_y
0,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,8,consulado,311.900114,Metro Alonso Martínez,"Plaza de Alonso Martínez nº 6,",1,0,24,7,15,-3.695440,40.427868
1,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,10,consulado,319.381126,Marqués de la Ensenada,"Calle Marqués de la Ensenada (junto a edif, Ce...",1,0,23,16,7,-3.692104,40.425403
2,Consulado de Argentina,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,104,consulado,380.025843,Castellana,"Paseo de la Castellana nº 4,",1,0,23,15,8,-3.689580,40.426770
3,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,148,consulado,255.765517,Serrano 210,"Calle Serrano nº 210,",1,0,23,12,11,-3.681397,40.451117
4,Consulado de Belice,CALLE TALAVERA 9,40.453376,-3.680978,159,consulado,312.602762,Paseo de la Habana 63,"Paseo de la Habana nº 63,",1,0,23,3,20,-3.683593,40.454385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,534,embajada,422.553954,"Mesena, 102","Calle Mesena, 102 ,",1,0,24,10,14,-3.663406,40.467724
467,Embajada de la República de Corea,CALLE GONZALEZ AMIGO 15,40.466107,-3.666833,285,embajada,537.184597,Santa María Magdalena,"Calle Santa María Magdalena, 15 ,",1,0,23,0,23,-3.670600,40.463100
468,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,141,embajada,179.924919,Plaza de Emilio Castelar,"Paseo de la Castellana nº 43,",1,0,12,1,11,-3.689392,40.435405
469,Embajada del Nepal,CALLE GENERAL ORAA 3 PLANTA bajo izquierda,40.436031,-3.687905,142,embajada,291.887270,Bajo elevado de Eduardo Dato,"Paseo de la Castellana nº 42,",1,0,23,11,12,-3.687915,40.433409
